In [1]:
! pip install huggingface-hub==0.23.0
! pip install sentence-transformers==2.6.0
! pip install tokenizers==0.19.1
! pip install transformers==4.39.0
! pip install openai
! pip install --upgrade langchain
! pip install langchain-community
! pip install langchain-experimental
! pip install faiss-cpu tiktoken
! pip install langchain-community faiss-cpu langchain-openai tiktoken pypdf chromadb


Looking in indexes: https://sonatype.adfdata.net/repository/pypi_adf/simple, https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://sonatype.adfdata.net/repository/pypi_adf/simple, https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://sonatype.adfdata.net/repository/pypi_adf/simple, https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.39.0 requires tokenizers<0.19,>=0.14, but you have tokenizers 0.19.1 which is incompatible.
Looking in indexes: htt

In [1]:
import openai
import config
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
import pymysql as MySQLdb
import pandas as pd
import pymysql
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from sentence_transformers import CrossEncoder
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


Storing sample querys in the Vactor-Database

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
import tiktoken

embedding = OpenAIEmbeddings(api_key=config.API_KEY)

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loaders = TextLoader("example_text.txt")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 10,
    separators=["\n\n"]
)

splits = text_splitter.split_documents(loaders.load())
split_texts=[str(doc.page_content) for doc in splits]




In [4]:
import chromadb
from chromadb.utils.embedding_functions import create_langchain_embedding

ef = create_langchain_embedding(embedding)


In [5]:
chroma_client = chromadb.Client() # we create a chromadb client like we created for everyother
client = chromadb.PersistentClient(path="database") # This initializes a persistent client, meaning the data will be stored persistently on disk rather than in memory.

chroma_collection = chroma_client.create_collection("2025_sample_query1", embedding_function=ef) # This method creates a new collection within ChromaDB.
# any document added to this collection will be processed using the ef function to generate its embedding.

In [6]:

ids = [str(i) for i in range(len(split_texts))] #creating ids for all splits

chroma_collection.add(ids=ids, documents=split_texts)
chroma_collection.count() # verify that all the documents were added successfully.


23

Filtering Similar Query

In [7]:
def get_simialr_query(query,N=5):
    results = chroma_collection.query(query_texts=[query], n_results=N)
    retrieved_documents = results['documents'][0]
    return retrieved_documents
    
def get_reranked_simialr_query(query,retrieved_documents,K=3):
    information=""
    cross_encoder = CrossEncoder("BAAI/bge-reranker-v2-m3")
    pairs = [[query, d_i] for d_i in retrieved_documents]
    scores = cross_encoder.predict(pairs)
    new_ord=[o for o in np.argsort(scores)[::-1] ]
    i=0
    for n in new_ord:
        information +=retrieved_documents[n]
        i+=1
        if(i==K):
         break #taking the first 4 out of 8
    return information
 
    


In [8]:


def get_df(query,db_name='local_database.db'):
    conn = sqlite3.connect(db_name)

    df = None
    try:
        df = pd.read_sql_query(sql = query,con=conn)
    finally:
        conn.close()
    return df

In [9]:
def get_df(query,host='analytics-rds.adfdata.net',user='nsanthoshsivan',passwd='t8N8TjQLTc8gyThP',db='decision'):
    conn = pymysql.connect(host=host,user=user,passwd=passwd,db=db)
    df = None
    try:
        df = pd.read_sql_query(sql = query,con=conn)
    finally:
        conn.close()
    return df

PromptTemplate

In [10]:
from langchain import LLMChain, PromptTemplate

def get_Query_prompt_Tamplets(examples):
    Schema = """Table: DV_tables.Equity_Deck_V2_20241206
    Description: This table provides details about loans, including their cycle-wise repayment status, interest rates, delinquency behavior, and current loan status. It helps track loans from their disbursal to maturity or default.
    Columns:

    Cycle_No (INTEGER): Loan repayment cycle ( Ranges from 1-36 ).
    disbursal_date (DATE): The date on which loan was disbursed.
    due_dt (DATE): The due date for each cycle of a loan.
    int_rate (FLOAT): Original interest rate that was given to customers. 
    loan_amount (FLOAT): Total loan amount that was disbursed to the customer in the starting ( It is only assigned to cycle no = 1 ).
    LoanID (TEXT): Foreign key linking to the Cure_Dv table and cl_import.loan__Loan_Account__c table representing the unique loan id of a loan.
    APR (FLOAT) : Annual percentage rate, assigned to the customer at the start of the loan ( Starts from 0 ).
    payment_freq (TEXT) - Represents the repayment frequency of the loan ( Monthly, Semi Monthly, Bi-Weekly or Weekly ).
    PD_1 (INTEGER): Stores 0,1 or -99 which represents if a customer was post deliquent 1 day after loan payment due date cycle wise, if yes then 1, if no then 0 and if loan is not matured till that cycle then -99.
    PD_15 (INTEGER): Stores 0,1 or -99 which represents if a customer was post deliquent 15 day after loan payment due date cycle wise, if yes then 1, if no then 0 and if loan is not matured till that cycle then -99.
    PD_30 (INTEGER): Stores 0,1 or -99 which represents if a customer was post deliquent 30 day after loan payment due date cycle wise, if yes then 1, if no then 0 and if loan is not matured till that cycle then -99.
    PD_120 (INTEGER): Stores 0,1 or -99 which represents if a customer was post deliquent 120 day after loan payment due date cycle wise, if yes then 1, if no then 0 and if loan is not matured till that cycle then -99.
    Status (TEXT): Stores Paid,Charged-Off,Deliquent,Active-Good and Active-Bad based on their current status cycle-wise.

    Relationships:
    Joinable With:
    cl_import.loan__Loan_Account__c on DV_tables.Equity_Deck_V2_20241206.LoanID = cl_import.loan__Loan_Account__c.Name.

    -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    Table: cl_import.loan__Loan_Account__c
    Description: This table maintains comprehensive payment details, delinquency tracking, and refinance status for each loan.
    Columns:
    
    Name (TEXT): Foreign key linking to the DV_tables.Equity_Deck_V2_20241206 table and cl_import.loan__Loan_Account__c table representing the unique loan id of a loan.
    loan__Loan_Amount__c (FLOAT): Total loan amount that was disbursed to the customer in the starting ( It is only assigned to cycle no = 1 ).
    loan__Fees_Paid__c (FLOAT): Represents the fees paid total for the loan.
    loan__Principal_Paid__c (FLOAT): Represents the total principal paid  for the loan.
    loan__Interest_Paid__c (FLOAT): Represents the total interest component paid for the loan.
    loan__Total_Pre_Paid_Fees__c (FLOAT): Represents the principal component paid for the loan.
    Deliquent_Days (INTEGER): Stores the delinquency days post due date for each cycle (Ranges from 0 to 150 ).
    Chargeoff_Amount (FLOAT): Represents the principal component which was left when the loan gets default at any cycle. ( Suppose Loan get default in 7th cycle, then at 7th cycle whatever principal remains will count as charged-off amount and it will be shown under this column in front of 7th cycle, after 7th cycle the loan will be closed and we won't generate any cycle for that Loan ID), A loan gets charged-off when its post delinquencies reaches 120 days after due date for each cycle.
    Recoveries_Paid (FLOAT): Represents the recoveries paid for the loan ( it is assigned to the last cycle number till which loan was active or last cycle no where loan didn't go charged-off), usually 5% of the charged-off amount.
    Refi_Flag (INTEGER): Stores 0 or 1, if a loan is a refinanced loan or not.

    Relationships:
    Joinable With:
    DV_tables.Equity_Deck_V2_20241206 on cl_import.loan__Loan_Account__c.Name = DV_tables.Equity_Deck_V2_20241206.LoanID."""


    message = f"""
            You are an expert SQL generator. 

            Your task is to generate efficient SQL queries based on the user's request that can be executed directly on a MySQL server.apply all required contitions according to the requirment and  Follow these guidelines while crafting the SQL queries:

            1. **Database Schema**: Assume the following schema is available:
            
            {Schema}
            
            
            2. **Terminology**:
            - **New Customers**: Customers visiting for the first time from `PS`, `ORG`, or `AFF` channels.
            - **Returning Customers**: Customers visiting for the first time from `RF` or `RA` channels.
            - **MM (Money Multipule)**: Calculated as `(Principal_Paid + Interest_Paid)/loan_amount`.
            - **Vintage**: The loan's disbursal month (`mm-yyyy`), which can be derived from the `Disbursal_Date` field.
            - **Due Month**: The due month (`mm-yyyy`) for different payment cycles of a loan, derived from the `Due_Date` field.
            - **XPDY** : Y day delinquency at cycle X, so get the PD_Y and cycle_X accordingly eg 6PD_15 means consider cycle= 6 and PD_15 flag

            3. **Examples**:
            {examples}
            
            
            4. **Key Rules**:
            - If multiple steps or queries are required to generate the desired result, use **Common Table Expressions (CTEs)** to ensure modularity and readability.
            - If a query involves altering the database (e.g., `INSERT`, `UPDATE`, `DELETE`), respond with **'Not allowed'** and provide an explanation if needed.

            5. **Expected Output**:
            - Provide a single, optimized SQL query using CTEs if necessary.
            - Ensure that the query adheres to best practices for MySQL syntax and performance optimization
            - **Only return the SQL query. Do not provide any additional explanation or context.**
            - Always order according to recent time data first and always limit to top 15 values only
        Now, generate the required SQL queries for by looking all the examples and the schema:
    
        """
    # Define the prompt template
    query_template = PromptTemplate(input_variables=["user_text"],template=message+"""{user_text}""")
    return query_template

# Define the prompt template
def get_insight_prompt_template():
    summarization_template = PromptTemplate(
        input_variables=["combined_input"],  # Combine inputs into a single key
        template="""
            You are an expert data presenter. Here is a dataset and query:
            {combined_input}

            Please provide the contents in a well-readable table format with a concise summary highlighting key insights and patterns in point form.
        """
    )
    return summarization_template

# Prepare the inputs
def format_inputs(dataframe, query_text):
    combined_input = f"""
    Dataset:
    {dataframe}

    Query:
    {query_text}
    """
    return combined_input

LLM Chains

In [11]:
# Define a function to generate the generate using the template
def generate_query(query,llm,query_template,memory):
    # Initialize the LLMChain with the prompt template
    chain = LLMChain(llm=llm, prompt=query_template,memory=memory
)
    
    # Run the chain with the provided product details
    description = chain.invoke({"user_text": query})

    return description

In [12]:
def generate_insight(SQL_query,user_text,llm,memory):
    forbidden_operations = ["insert", "update", "delete"]
    if any(op in SQL_query.lower() for op in forbidden_operations):
        Insights = "Not allowed: Query contains forbidden operations (INSERT, UPDATE, DELETE)."
    else:
        temp = get_df(SQL_query).to_string(index=False)
        combined_input = format_inputs(temp, query_text)

        summarization_template = get_insight_prompt_template()
        # Define the LangChain for summarization
        summarization_chain = LLMChain(llm=llm, prompt=summarization_template,memory=memory)
        
        # Generate the summary
        Insights = summarization_chain.run(combined_input=combined_input)
    return Insights


Inferance

In [16]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0 or 0.2
llm = ChatOpenAI(openai_api_key=config.API_KEY,temperature=0.2, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()

query_text = "What was 3PD_30 rate for different vintage?"   
print("User Query:-")
print("----------------")
print(query_text,"\n")

doc = get_simialr_query(query_text,N=10)
examples = get_reranked_simialr_query(query_text,doc,K=5)

query_template= get_Query_prompt_Tamplets(examples)
sql_query = generate_query(query_text,llm,query_template,memory)['text']
print("Generated SQL Query:-")
print("----------------")
print(sql_query,"\n")


Insights = generate_insight(sql_query,query_text,llm,memory)
print("Insights:-")
print("----------------")
print(Insights,"\n")


User Query:-
----------------
What was 3PD_30 rate for different vintage? 



Generated SQL Query:-
----------------
WITH CTE AS (
    SELECT LEFT(disbursal_date, 7) AS Vintage, 
           (SUM(PD_30) / COUNT(PD_30)) AS PD30_Rate 
    FROM DV_tables.Equity_Deck_V2_20241206 
    WHERE PD_30 != -99 AND cycle_no=3 
    GROUP BY 1
)
SELECT * FROM CTE
ORDER BY Vintage DESC
LIMIT 15; 

Insights:-
----------------
| Vintage | PD30_Rate |
|---------|-----------|
| 2024-08 | 0.105540  |
| 2024-07 | 0.104837  |
| 2024-06 | 0.104117  |
| 2024-05 | 0.108900  |
| 2024-04 | 0.096163  |
| 2024-03 | 0.096636  |
| 2024-02 | 0.081258  |
| 2024-01 | 0.080731  |
| 2023-12 | 0.082171  |
| 2023-11 | 0.079568  |
| 2023-10 | 0.085927  |
| 2023-09 | 0.082079  |
| 2023-08 | 0.086752  |
| 2023-07 | 0.090507  |
| 2023-06 | 0.095048  |

- Key Insights:
  1. PD30 rates vary over different vintage periods.
  2. There is a general fluctuation in PD30 rates over time.
  3. The PD30 rate for 2024-05 is notably higher compared to other months.
  4. The PD30 rate for 2023-11 is one of the lowest 

In [32]:
query_text = "what are the Intrest_rate and Loan_amount varies accros vintages"   
print("User Query:-")
print("----------------")
print(query_text,"\n")
doc = get_simialr_query(query_text)
examples = get_reranked_simialr_query(query_text,doc)

query_template= get_Query_prompt_Tamplets(examples)
sql_query = generate_query(query_text,llm,query_template,memory)['text']
print("Generated SQL Query:-")
print("----------------")
print(sql_query,"\n")

Insights = generate_insight(sql_query,query_text,llm,memory)
print("Insights:-")
print("----------------")
print(Insights,"\n")


User Query:-
----------------
what are the Intrest_rate and Loan_amount varies accros vintages 



Generated SQL Query:-
----------------
WITH Vintage_Data AS (
    SELECT LEFT(loan__Disbursal_Date__c, 7) AS Vintage, 
           AVG(loan__Interest_Rate__c) AS Average_Int_Rate,
           AVG(loan__Loan_Amount__c) AS Average_Loan_Amount
    FROM cl_import.loan__Loan_Account__c
    GROUP BY 1
)
SELECT Vintage, Average_Int_Rate, Average_Loan_Amount
FROM Vintage_Data
ORDER BY Vintage DESC
LIMIT 15; 

Insights:-
----------------
| Vintage | Average_Int_Rate | Average_Loan_Amount |
|---------|------------------|---------------------|
| 2025-01 | 94.057533        | 3612.338539         |
| 2024-12 | 96.322813        | 3713.075650         |
| 2024-11 | 94.480670        | 3956.373681         |
| 2024-10 | 95.580817        | 3879.849301         |
| 2024-09 | 96.581338        | 3769.669990         |
| 2024-08 | 94.174585        | 3983.505230         |
| 2024-07 | 92.078686        | 3970.234204         |
| 2024-06 | 92.121457        | 3942.135834         |
| 2024-05 | 90.195012        | 3984.504

In [33]:
query_text = "what is the MM for different vintages"   
print("User Query:-")
print("----------------")
print(query_text,"\n")

doc = get_simialr_query(query_text)
examples = get_reranked_simialr_query(query_text,doc)

query_template= get_Query_prompt_Tamplets(examples)
sql_query = generate_query(query_text,llm,query_template,memory)['text']
print("Generated SQL Query:-")
print("----------------")
print(sql_query,"\n")

Insights = generate_insight(sql_query,query_text,llm,memory)
print("Insights:-")
print("----------------")
print(Insights,"\n")



User Query:-
----------------
what is the MM for different vintages 



Generated SQL Query:-
----------------
WITH MM_Calculation AS (
    SELECT LEFT(loan__Disbursal_Date__c, 7) AS Vintage, 
        (SUM(loan__Principal_Paid__c) + SUM(loan__Interest_Paid__c)) / SUM(loan__Loan_Amount__c) AS MM 
    FROM cl_import.loan__Loan_Account__c 
    GROUP BY LEFT(loan__Disbursal_Date__c, 7)
)

SELECT Vintage, MM
FROM MM_Calculation
ORDER BY Vintage DESC
LIMIT 15; 

Insights:-
----------------
| Vintage | MM      |
|---------|---------|
| 2025-01 | 0.000365|
| 2024-12 | 0.057688|
| 2024-11 | 0.139062|
| 2024-10 | 0.226771|
| 2024-09 | 0.314799|
| 2024-08 | 0.396743|
| 2024-07 | 0.521825|
| 2024-06 | 0.606761|
| 2024-05 | 0.660751|
| 2024-04 | 0.730173|
| 2024-03 | 0.791842|
| 2024-02 | 0.875584|
| 2024-01 | 0.920610|
| 2023-12 | 0.976834|
| 2023-11 | 0.966970|

Key insights and patterns:
- The MM values generally increase over time, indicating a positive trend.
- There is a slight decrease in MM from 2023-12 to 2024-01, followed by a steady increase.
- The MM values

In [18]:
query_text = "What are the vintages where 3PD_30 between 7% and 9% and what is the MM for that vintages?"   
print("User Query:-")
print("----------------")
print(query_text,"\n")

doc = get_simialr_query(query_text)
examples = get_reranked_simialr_query(query_text,doc)

query_template= get_Query_prompt_Tamplets(examples)
sql_query = generate_query(query_text,llm,query_template,memory)['text']
print("Generated SQL Query:-")
print("----------------")
print(sql_query,"\n")

Insights = generate_insight(sql_query,query_text,llm,memory)
print("Insights:-")
print("----------------")
print(Insights,"\n")



User Query:-
----------------
What are the vintages where 3PD_30 between 7% and 9% and what is the MM for that vintages? 



Generated SQL Query:-
----------------
WITH PD_30_Rates AS (
    SELECT LEFT(a.disbursal_date, 7) AS Vintage, 
           (SUM(a.PD_30) / COUNT(a.PD_30)) AS PD30_Rate,
           (SUM(b.loan__Principal_Paid__c) + SUM(b.loan__Interest_Paid__c)) / SUM(b.loan__Loan_Amount__c) AS MM
    FROM DV_tables.Equity_Deck_V2_20241206 a
    JOIN cl_import.loan__Loan_Account__c b ON a.LoanID = b.Name
    WHERE a.PD_30 != -99
    GROUP BY 1
)

SELECT Vintage, MM
FROM PD_30_Rates
WHERE PD30_Rate BETWEEN 0.07 AND 0.09
ORDER BY Vintage DESC
LIMIT 15; 

Insights:-
----------------
| Vintage | MM     |
|---------|--------|
| 2024-07 | 0.534134 |

Summary:
- The vintage 2024-07 has a MM value of 0.534134.
- There are no vintages where 3PD_30 is between 7% and 9% in the provided dataset.
- The dataset does not contain any vintages meeting the specified criteria. 



In [30]:
print("SQL Generator Chat! Type 'exit' to quit.")
forbidden_operations = ["insert", "update", "delete"]
llm = ChatOpenAI(openai_api_key=config.API_KEY,temperature=0.2, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()

query_text = "compare the average APR for PS and RF Channel"


while True:
    query_text = input("You: ")
    if query_text.lower() == "exit":
        break
    try:
        print("User Query:-")
        print("----------------")
        print(query_text,"\n")

        doc = get_simialr_query(query_text,N=8)
        examples = get_reranked_simialr_query(query_text,doc,K=4)

        query_template= get_Query_prompt_Tamplets(examples)
        sql_query = generate_query(query_text,llm,query_template,memory)['text']
        print("Generated SQL Query:-")
        print("----------------")
        print(sql_query,"\n")

        Insights = generate_insight(sql_query,query_text,llm,memory)
        print("Insights:-")
        print("----------------")
        print(Insights,"\n")

    except Exception as e:
        print(f"An error occurred: {e}")

SQL Generator Chat! Type 'exit' to quit.


User Query:-
----------------
compare the average APR for PS and RF Channel 

Generated SQL Query:-
----------------
WITH PS_APR AS (
    SELECT AVG(APR) AS avg_apr_ps
    FROM DV_tables.Equity_Deck_V2_20241206 a 
    JOIN cl_import.loan__Loan_Account__c b ON a.LoanID = b.Name
    WHERE Lead_Path__c = 'PS'
),
RF_APR AS (
    SELECT AVG(APR) AS avg_apr_rf
    FROM DV_tables.Equity_Deck_V2_20241206 a 
    JOIN cl_import.loan__Loan_Account__c b ON a.LoanID = b.Name
    WHERE Lead_Path__c = 'RF'
)
SELECT avg_apr_ps, avg_apr_rf
FROM PS_APR, RF_APR; 

Insights:-
----------------
| Channel | Average APR |
|---------|-------------|
| PS      | 101.95557   |
| RF      | 76.052767   |

Summary:
- The average APR for the PS channel is significantly higher than the RF channel.
- There is a notable difference of approximately 25.9 between the average APR for PS and RF channels.
- The PS channel appears to have a higher average APR compared to the RF channel. 



Exploring Free-to-Use Open-Source LLMs:

In [21]:
!pip install langchain_ollama

Looking in indexes: https://sonatype.adfdata.net/repository/pypi_adf/simple, https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [33]:
!pip install langchain_ollama
ollama pull mistral:v0.3
!pip install -U langchain-ollama

Looking in indexes: https://sonatype.adfdata.net/repository/pypi_adf/simple, https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [41]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#############                                            42.4% 64.0%                65.0%######                    76.2%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
Created symlink /etc/systemd/system/default.target.wants/ollama.service → /etc/systemd/system/ollama.service.
>>> NVIDIA GPU installed.


In [48]:
!ollama pull llama3.2:3b-instruct-q8_0


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ 

pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏    0 B/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏    0 B/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏    0 B/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏ 608 KB/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏ 2.4 MB/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏ 5.2 MB/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   0% ▕                ▏  12 MB/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   1% ▕                ▏  25 MB/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   1% ▕                ▏  31 MB/3.4 GB                  pulling manifest 
pulling ed5cd7dbde6e...   2% ▕   

In [34]:
from langchain_ollama import ChatOllama
#from langchain_ollama import OllamaLLM

llm = ChatOllama(model="llama3.2:3b-instruct-q8_0")
test_prompt = "What is the capital of France?"

# Invoke the LLM and print the result
response = llm.invoke(test_prompt)
print("Response:")
print(response)


Response:
content='The capital of France is Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:3b-instruct-q8_0', 'created_at': '2025-01-05T15:28:51.961923066Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4005804163, 'load_duration': 2556435041, 'prompt_eval_count': 32, 'prompt_eval_duration': 735000000, 'eval_count': 8, 'eval_duration': 713000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-394a9605-6249-4ffd-8928-538e99f1518b-0' usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40}


In [43]:
query_text =  "compare the average APR for PS and RF Channel"
doc = get_simialr_query(query_text,N=8)
examples = get_reranked_simialr_query(query_text,doc,K=4)


In [46]:
#uery_template= get_Query_prompt_Tamplets(examples)
#sql_query = generate_query(query_text,llm,query_template,memory)['text']
print("User Query:-")
print("----------------")
print(query_text,"\n")

print("Generated SQL Query:-")
print("----------------")
print(sql_query,"\n")
Insights = generate_insight(sql_query,query_text,llm,memory)
print("Insights:-")
print("----------------")
print(Insights,"\n")



User Query:-
----------------
compare the average APR for PS and RF Channel 

Generated SQL Query:-
----------------
WITH CTE AS (
    SELECT 
        a.APR,
        b.Lead_Path__c,
        a.disbursal_date,
        EXTRACT(MONTH FROM a.disbursal_date) AS month,
        EXTRACT(YEAR FROM a.disbursal_date) AS year,
        b.loan__Loan_Amount__c
    FROM DV_tables.Equity_Deck_V2_20241206 a 
    JOIN cl_import.loan__Loan_Account__c b ON a.loanid = b.Name
    WHERE b.Lead_Path__c IN ('PS', 'RF')
)
SELECT 
    LEAD_PATH__c,
    AVG(APR) AS avg_APR
FROM CTE
GROUP BY LEAD_PATH__c 



Insights:-
----------------
**Dataset Analysis: Average APR Comparison**

| **Channel** | **Average APR** |
| --- | --- |
| PS | 101.955570 |
| RF | 76.052767 |

**Summary and Insights:**

* The average APR for the PS Channel is significantly higher than that of the RF Channel, indicating potential cost savings or more favorable interest rates for RF customers.
* A comparison of the two channels suggests that PS may be a less attractive option for borrowers due to its higher APR.
* This could have implications for business strategy and marketing efforts, as PS may need to consider competitive pricing adjustments to remain competitive in the market.

**Key Takeaways:**

• The PS Channel has a significantly higher average APR compared to the RF Channel.
• Borrowers may benefit from opting for the RF Channel due to its lower APR.
• Further analysis is needed to determine if these differences are statistically significant or if there are other factors at play. 

